# <font color=#531AC8>Fatoração de Matrizes para Sistemas de Classificação de Machine Learning</font>
> __Trabalho para a disciplina CCM0218 (2020.2)__ <br>
Fernando Valls Yoshida (11246714) e Lucas de Sousa Rosa (11296717)

Para a realização deste trabalho, nos reunimos periodicamente para ler o material fornecido, acompanhando as passagens e discutindo os pseudo-códigos. Enquanto isso, foi elaborado este Jupyter Notebook, em que constam os códigos que escrevemos, as considerações sobre eles e o relatório das tarefas exigidas. Na ordem natural em que aparecem no texto fornecido, os temas abordados virão neste documento.

__Seção para importação das bibliotecas__

In [72]:
import numpy as np
import math
import matplotlib.pyplot as plt
import time

## <font color=#C8531A>Primeira tarefa</font>

> Observação: em Álgebra Linear, é natural contar as posições de matrizes começando em 1, mas Numpy o faz começando em 0. Em face disso, aqui optou-se por manter a contagem da Álgebra sempre que possível. Em caso contrário, são feitas intervenções diretas no sistema de contagem, ou como ajuste geral (fazendo, e.g., $i = i - 1$ no início de um trecho de código) ou como ajuste de indexação no uso de um elemento da matriz (e.g., $W[i - 1, j - 1]$ para fazer $W_{i,j}$). Sempre que conveniente, tais medidas são apontadas no corpo do código.

A primeira tarefa trata da __resolução de sistemas lineares através da Fatoração QR__, em caso de sistemas sistemas sobredeterminados, individual (`solve`) e simultaneamente (`solve_multi`). Para ambos os casos, faz-se necessário implementar uma rotação de Givens $Q(i,j,\theta)$ (`rot_Givens`) que, por sua vez, depende de uma implementação de `cos_sin`, definido tal como no documento de referência.<br>
Estes quatro métodos encontram-se abaixo. As verificações de funcionamento de `cos_sin` e `rot_Givens` estão no __Apêndice__ deste documento.

__Determina os valores de cos(θ) e sin(θ), em conformidade com (3) e (4)__

In [2]:
def cos_sin(W, i, j, k):
    # ajuste de contagem
    i = i - 1
    j = j - 1
    k = k - 1
    
    if abs(W[i,k]) > abs(W[j,k]):
        t = -W[j,k]/W[i,k]
        c = 1/math.sqrt(1 + t**2)
        s = c*t
    else:
        t = -W[i, k]/W[j, k]
        s = 1/math.sqrt(1 + t**2)
        c = s*t    
    return c, s

__Rotação de Givens Q(i, j, θ) a ser aplicada em $W_{n \times m}$__

In [3]:
def rot_Givens(W, n, m, i, j, c, s):
    # ajuste de contagem
    i = i - 1
    j = j - 1
    
    for r in range(m):
        aux = c*W[i,r] - s*W[j,r]
        W[j,r] = s*W[i,r] + c*W[j,r]
        W[i,r] = aux

__Resolução de sistemas sobredeterminados__

In [4]:
def solve(W, b, n, m):
    eps = 10e-10
    for k in range(1, m + 1):
        for j in range(n, k, -1):
            i = j - 1
            if abs(W[j - 1, k - 1]) > eps:
                cos, sin = cos_sin(W, i, j, k)
                rot_Givens(W, n, m, i, j, cos, sin)
                rot_Givens(b, n, 1, i, j, cos, sin)
    
    for k in range(m, 0, -1):
        soma = 0
        for j in range(k + 1, m + 1):
            soma += W[k - 1, j - 1]*b[j - 1, 0]
        b[k - 1, 0] = (b[k - 1, 0] - soma)/W[k - 1, k - 1]

__Vários sistemas simultâneos__

In [5]:
def solve_multi(W, A, n, m, p):
    eps = 10e-10
    for k in range(1, p + 1):
        for j in range(n, k, -1):
            i = j - 1
            if abs(W[j - 1, k - 1]) > eps:
                cos, sin = cos_sin(W, i, j, k)
                rot_Givens(W, n, p, i, j, cos, sin)
                rot_Givens(A, n, m, i, j, cos, sin)
    
    for k in range(p, 0, -1):
        for j in range(1, m + 1):
            soma = 0
            for i in range(k+1,p+1):
                soma += W[k-1,i-1]*A[i-1,j-1]
            A[k-1,j-1] = (A[k-1,j-1] - soma)/W[k-1,k-1]

## Testes sugeridos

In [6]:
def teste_a():
    n = 64
    m = 64
    W = np.zeros((n,m))
    b = np.zeros((n,1))
    for i in range(1, n+1):
        for j in range(1, m+1):
            if i == j:
                W[i - 1, j - 1] = 2
            elif abs(i - j) == 1:
                W[i - 1, j - 1] = 1
            elif abs(i - j) > 1:
                W[i - 1, j - 1] = 0
            b[i - 1, 0] = 1
    solve(W,b,n,m)
    print(b)
teste_a()

[[0.49230769]
 [0.01538462]
 [0.47692308]
 [0.03076923]
 [0.46153846]
 [0.04615385]
 [0.44615385]
 [0.06153846]
 [0.43076923]
 [0.07692308]
 [0.41538462]
 [0.09230769]
 [0.4       ]
 [0.10769231]
 [0.38461538]
 [0.12307692]
 [0.36923077]
 [0.13846154]
 [0.35384615]
 [0.15384615]
 [0.33846154]
 [0.16923077]
 [0.32307692]
 [0.18461538]
 [0.30769231]
 [0.2       ]
 [0.29230769]
 [0.21538462]
 [0.27692308]
 [0.23076923]
 [0.26153846]
 [0.24615385]
 [0.24615385]
 [0.26153846]
 [0.23076923]
 [0.27692308]
 [0.21538462]
 [0.29230769]
 [0.2       ]
 [0.30769231]
 [0.18461538]
 [0.32307692]
 [0.16923077]
 [0.33846154]
 [0.15384615]
 [0.35384615]
 [0.13846154]
 [0.36923077]
 [0.12307692]
 [0.38461538]
 [0.10769231]
 [0.4       ]
 [0.09230769]
 [0.41538462]
 [0.07692308]
 [0.43076923]
 [0.06153846]
 [0.44615385]
 [0.04615385]
 [0.46153846]
 [0.03076923]
 [0.47692308]
 [0.01538462]
 [0.49230769]]


In [7]:
def teste_b():
    n = 20
    m = 17
    W = np.zeros((n,m))
    b = np.zeros((n,1))
    
    for i in range(1,n+1):
        for j in range(1,m+1):
            if abs(i - j) <= 4:
                W[i - 1, j - 1] = 1.0/(i + j - 1)
            elif abs(i - j) > 4:
                W[i - 1, j - 1] = 0
        b[i - 1, 0] = i
    
    solve(W, b, n, m)
    print(b)
    
teste_b()

[[ 56.35780802]
 [-45.87484885]
 [-43.48902374]
 [-48.5765491 ]
 [-30.14020001]
 [ 89.81211894]
 [ 48.713648  ]
 [ 59.23924714]
 [ 11.44635697]
 [109.16273559]
 [-72.87275483]
 [-54.36289269]
 [-51.44441718]
 [-25.01481463]
 [ 98.57193882]
 [218.65885869]
 [298.40022751]
 [ -1.87505936]
 [ 10.96948262]
 [ -0.7493688 ]]


In [8]:
def teste_c():
    n = 64
    m = 3
    p = 64
    W = np.zeros((n,p))
    A = np.zeros((n,m))
    
    for i in range(1,n+1):
        for j in range(1, p+1):
            if abs(i - j) == 1:
                W[i - 1, j - 1] = 1
            elif abs(i - j) > 1:
                W[i - 1, j - 1] = 0
        A[i - 1, 0] = 1
        A[i - 1, 1] = i
        A[i - 1, 2] = 2*i
    print(W)
    solve_multi(W,A,n,m,p)
    print(A)

teste_c()

[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]
[[ -0. -32. -64.]
 [  1.   1.   2.]
 [  1.  34.  68.]
 [ -0.   2.   4.]
 [ -0. -30. -60.]
 [  1.   3.   6.]
 [  1.  36.  72.]
 [ -0.   4.   8.]
 [ -0. -28. -56.]
 [  1.   5.  10.]
 [  1.  38.  76.]
 [ -0.   6.  12.]
 [ -0. -26. -52.]
 [  1.   7.  14.]
 [  1.  40.  80.]
 [ -0.   8.  16.]
 [ -0. -24. -48.]
 [  1.   9.  18.]
 [  1.  42.  84.]
 [ -0.  10.  20.]
 [ -0. -22. -44.]
 [  1.  11.  22.]
 [  1.  44.  88.]
 [ -0.  12.  24.]
 [ -0. -20. -40.]
 [  1.  13.  26.]
 [  1.  46.  92.]
 [ -0.  14.  28.]
 [ -0. -18. -36.]
 [  1.  15.  30.]
 [  1.  48.  96.]
 [ -0.  16.  32.]
 [ -0. -16. -32.]
 [  1.  17.  34.]
 [  1.  50. 100.]
 [ -0.  18.  36.]
 [ -0. -14. -28.]
 [  1.  19.  38.]
 [  1.  52. 104.]
 [ -0.  20.  40.]
 [ -0. -12. -24.]
 [  1.  21.  42.]
 [  1.  54. 108.]
 [ -0.  22.  44.]
 [ -0. -10. -20.]
 [  1.  23.  46.]
 [  1.  56. 112.

In [9]:
def teste_d():
    n = 20
    m = 3
    p = 17
    W = np.zeros((n,p))
    A = np.zeros((n,m))
    
    for i in range(1,n+1):
        for j in range(1,p+1):
            if abs(i - j) <= 4:
                W[i - 1, j - 1] = 1.0/(i + j - 1)
            elif abs(i - j) > 4:
                W[i - 1, j - 1] = 0
        A[i - 1, 0] = 1
        A[i - 1, 1] = i
        A[i - 1, 2] = 2*i
    solve_multi(W,A,n,m,p)
    print(A)
teste_d()

[[ 2.88155063e+00  5.63578080e+01  1.12715616e+02]
 [-1.83376253e+00 -4.58748489e+01 -9.17496977e+01]
 [-1.51398904e+00 -4.34890237e+01 -8.69780475e+01]
 [-1.52190762e+00 -4.85765491e+01 -9.71530982e+01]
 [-4.53794614e-01 -3.01402000e+01 -6.02804000e+01]
 [ 5.85669889e+00  8.98121189e+01  1.79624238e+02]
 [ 3.42192891e+00  4.87136480e+01  9.74272960e+01]
 [ 3.65656219e+00  5.92392471e+01  1.18478494e+02]
 [ 1.20368454e+00  1.14463570e+01  2.28927139e+01]
 [ 6.12534249e+00  1.09162736e+02  2.18325471e+02]
 [-2.47971396e+00 -7.28727548e+01 -1.45745510e+02]
 [-1.47793147e+00 -5.43628927e+01 -1.08725785e+02]
 [-1.20390258e+00 -5.14444172e+01 -1.02888834e+02]
 [ 1.28414690e-01 -2.50148146e+01 -5.00296293e+01]
 [ 6.50158900e+00  9.85719388e+01  1.97143878e+02]
 [ 1.14910560e+01  2.18658859e+02  4.37317717e+02]
 [ 1.45805258e+01  2.98400228e+02  5.96800455e+02]
 [-8.48567880e-02 -1.87505936e+00 -3.75011871e+00]
 [ 5.30540939e-01  1.09694826e+01  2.19389652e+01]
 [-3.29633180e-02 -7.49368799e-

## <font color=#C8531A>Segunda tarefa</font>

__Determina o erro quadrático $E = \| A - WH  \|^2 $__

In [10]:
def erro(A,W,H,n,m):
    E = 0
    prod = W.dot(H)
    for i in range(1,n+1):
        for j in range(1,m+1):
            E += (A[i-1,j-1] - prod[i-1,j-1])**2
    return E

__Normaliza W de tal modo que $w_{i,j} = \frac{w_{i,j}}{s_{j}}$, com $s_{j} = \sqrt{\sum_{i=1}^{n}w_{i,j}^2}$__

In [11]:
def norm(W,n,p):
    for j in range(1,p+1):
        s = 0
        for i in range(1,n+1):
            s += (W[i-1,j-1])**2
        s = math.sqrt(s)
        for i in range(1,n+1):
            W[i-1,j-1] = W[i-1,j-1]/s
    return W

__Define $B_{p \times m}$ a partir de $A_{n \times m}$. B é definido a partir da porção superior (p primeiras linhas e todas as m colunas) de A, sendo ainda que $b_{i,j} = a_{i,j}$, se e somente se $a_{i,j} \geq 0$, do contrário $b_{i,j} = 0$__

In [12]:
def set_matrix(A,p,m):
    B = np.zeros((p,m))
    for i in range(1,p+1):
        for j in range(1,m+1):
            if A[i-1,j-1] > 0:
                B[i-1,j-1] = A[i-1,j-1]
    return B

__Fatoração por matrizes não-negativas__

In [13]:
def NMF(A,n,m,p):
    # parâmetros
    eps = 10e-5
    it_max = 100
    # inicialização de variáveis de apoio
    e1 = 0
    e2 = 0
    t = 0
    # inicialização/setagem das matrizes de interesse
    W = np.random.rand(n,p)
    A_copia = np.copy(A)
    At_copia = np.copy(np.transpose(A))
    H = np.zeros((p,m))
    
    while(t < it_max):
        if (t > 1 and abs(e1 - e2) < eps):
            break
    
        A = np.copy(A_copia)           # recuperar A original
        W = norm(W,n,p)                # normalizar W
        solve_multi(W, A, n, m, p)     # WH = A
        H = set_matrix(A,p,m)          # H a partir de A

        At = np.copy(At_copia)         # computar A^t
        Ht = np.copy(np.transpose(H))  # computar H^t
        solve_multi(Ht, At, m, n, p)   # (WH)^t = H^tW^t = A^t
        Wt = set_matrix(At, p, n)      # W^t a partir de A^t
        W = np.copy(np.transpose(Wt))  # computar W (a partir de W^t)
        
        if t % 2 != 0:                    # erro em iteração ímpar
            e1 = erro(A_copia, W, H, n, m)
        else:                             # erro em iteração par
            e2 = erro(A_copia, W, H, n, m)
        
        t += 1
    return W, H

## <font color=#C8531A>Tarefa principal</font>

In [15]:
def train_dig(digito, ndig_treino, p):
    n = 784
    A = np.loadtxt("dados_mnist/train_dig" + str(digito) + ".txt")
    A = A[:,:ndig_treino]
    m = ndig_treino
    Wd, H = NMF(A,n,m,p)
    return Wd

In [86]:
def train_all():
    ndig_treino = [100, 1000, 4000]
    ps = [5,10,15]

    for ndig in ndig_treino:
        for p in ps:
            for digito in range(10):
                start = time.time()
                Wd = train_dig(digito, ndig, p)
                elapsed_time_lc = (time.time() - start)
                filename = "dig_treinos/W_" + str(digito) + "_" + str(ndig) + "_" + str(p) + ".txt" 
                with open(filename) as f:
                    f.write(elapsed_time_lc)
                    np.savetxt(f, Wd)

In [84]:
mat = np.array([[1.0,2.0,3.0,4.0],[3.0,4.0,5.0,6.0]])
with open("testing.txt", "w+") as f:
    f.write("este é o tempo\n")
    np.savetxt(f, mat)

In [85]:
def display(col):
    img = np.zeros((28,28))
    for t in range(1,785):
        j = t // 28
        i = t - 28*j
        img[j - 1, i - 1] = col[t - 1, 0]

    plt.imshow(img, interpolation='nearest')

## <font color=#C8531A>Apêndice</font>

Os testes foram realizados executando os métodos em estudo em situações cujo resultado é previamente conhecido, seja porque fornecido no documento de referência, porque calculado manualmente e/ou no `Octave`, ou porque evidente. Em todo caso, tal conhecimento "a priori" é aqui disposto acima de cada célula de código do teste.

> Observação: os testes vêm como métodos `teste_X` em virtude do não-compartilhamento (perigoso) de variáveis cujo nome é comumente empregado em demais métodos (e.g., $W$, $A$ etc.).

### <font color=#C81A38>Teste dos métodos da Primeira Tarefa</font>

__Teste de `cos_sin`__

$W =
\begin{pmatrix}
1 & 2 \\
3 & 4
\end{pmatrix}$

$i = 1, j = 2, k = 1 \hspace{1cm} \left( w_{1,1} = 1 < 3 = w_{2,1} \right) \hspace{1cm} \tau = -\frac{w_{1,1}}{w_{2,1}} = -\frac{1}{3}\\
sin = \frac{1}{\sqrt{1 + \tau^{2}}} = \frac{1}{\sqrt{1 + (\frac{1}{3})^{2}}} \approx 0,9486833... \hspace{2cm} cos = \tau sin = - \frac{0,9486833}{3} \approx -0,31622777...$

In [18]:
def teste_cos_sin():
    W = np.array([[1.0,2.0], 
                  [3.0,4.0]])
    c, s = cos_sin(W, 1, 2, 1)
    print("cos:",c)
    print("sin:",s)
teste_cos_sin()

cos: -0.3162277660168379
sin: 0.9486832980505138


__Teste de `rot_Givens`__

Exemplo da página 2 do documento de referência.<br>
$Q(3,4,\theta) \cdot W = Q(3,4,\theta) \cdot
\begin{pmatrix}
2 & 1 & 1 & -1 & 1 \\
0 & 3 & 0 & 1 & 2 \\
0 & 0 & 2 & 2 & -1 \\
0 & 0 & -1 & 1 & 2 \\
0 & 0 & 0 & 3 & 1
\end{pmatrix} = 
\begin{pmatrix}
2 & 1 & 1 & -1 & 1 \\
0 & 3 & 0 & 1 & 2 \\
0 & 0 & \frac{5}{\sqrt{5}} & \frac{3}{\sqrt{5}} & -\frac{4}{\sqrt{5}} \\
0 & 0 & 0 & \frac{4}{\sqrt{5}} & \frac{3}{\sqrt{5}} \\
0 & 0 & 0 & 3 & 1
\end{pmatrix} \approx
\begin{pmatrix}
2 & 1 & 1 & -1 & 1 \\
0 & 3 & 0 & 1 & 2 \\
0 & 0 & 2,23606798 & 1,34164079 & -1,78885438 \\
0 & 0 & 0 & 1,78885438 & 1,34164079 \\
0 & 0 & 0 & 3 & 1
\end{pmatrix}$

In [21]:
def teste_rot_Givens():
    A = np.array([[2.0, 1.0, 1.0,-1.0, 1.0],
                  [0.0, 3.0, 0.0, 1.0, 2.0],
                  [0.0, 0.0, 2.0, 2.0,-1.0],
                  [0.0, 0.0,-1.0, 1.0, 2.0],
                  [0.0, 0.0, 0.0, 3.0, 1.0]])

    cos, sin = cos_sin(A, 3, 4, 3)
    rot_Givens(A, 5, 5, 3, 4, cos, sin)
    print(A)
teste_rot_Givens()

[[ 2.          1.          1.         -1.          1.        ]
 [ 0.          3.          0.          1.          2.        ]
 [ 0.          0.          2.23606798  1.34164079 -1.78885438]
 [ 0.          0.          0.          1.78885438  1.34164079]
 [ 0.          0.          0.          3.          1.        ]]


__Teste de `solve`__

In [ ]:
W = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = np.array([[1.0], [1.0]])

solve(W, b, 2, 2)

print("final \n{}".format(b))

__Teste de `solve_multi`__

In [ ]:
W = np.array([[1.0, 2.0], [2.0, 1.0]])
A = np.array([[1.0,2.0,3.0],[4.0,5.0,6.0]])

solve_multi(W, A, 2, 3, 2)
print("final\n{}".format(A))

### <font color=#C81A38>Teste dos métodos da Segunda Tarefa</font>

__Teste da fatoração por matrizes não-negativas (com eps = 10e-40 pro resultado sair bonitinho rsrs)__

In [26]:
A = np.array([[0.3,0.6,0.0],[0.5,0.0,1.0],[0.4,0.8,0.0]])
n = 3
m = 3
p = 2
W, H = NMF(A, n, m, p)
print("A:\n",A)
print("W:\n",W)
print("H:\n",H)

A:
 [[0.3 0.6 0. ]
 [0.5 0.  1. ]
 [0.4 0.8 0. ]]
W:
 [[6.00121500e-01 1.21648158e-04]
 [0.00000000e+00 9.99999261e-01]
 [8.00162000e-01 1.62197544e-04]]
H:
 [[0.49939201 1.         0.        ]
 [0.50000037 0.         1.00000074]]
